
Environment Setup and Library Imports:
The code begins with setting up the environment and importing the necessary libraries and packages. It installs required packages and imports libraries such as PyTorch, PyTorch Geometric (PyG), pandas, numpy, and more.

Dataset and Preprocessing:

The MovieLens 100K dataset is loaded into a DataFrame df1.
The dataset is filtered to keep only high-rated interactions (ratings of 3 or above).
The dataset is split into train and test sets, and user and item labels are relabeled to ensure uniqueness.
Minibatch Sampling (data_loader function):

This function samples positive and negative items for users in each minibatch.
It takes the train DataFrame, batch size, number of users, and number of items as inputs.
It samples positive items randomly from the user's interactions and negative items not present in their interactions.
Edge Index Generation (train_edge_index):

The edge index is generated for the bipartite graph based on user-item interactions.
Each user is connected to the items they have interacted with, and vice versa.
LightGCN Convolutional Layer (LightGCNConv class):

The LightGCN convolutional layer is defined by inheriting from PyG's MessagePassing class.
The forward method computes normalization factors and propagates messages.
The message method computes the normalized messages for aggregation.
NGCF Layer (NGCFConv class):

The NGCF convolutional layer is defined similarly to the LightGCNConv class.
It includes linear layers and activation functions for message aggregation.
Dropout is applied to the output before the activation.
Recommender System GNN (RecSysGNN class):

This class stacks the specified number of convolutional layers (NGCF or LightGCN).
It includes methods for forward propagation and encoding minibatches of users and items.
Loss Function and Metrics:

The Bayesian Personalized Ranking (BPR) loss function is implemented.
The precision@K and recall@K metrics are computed for evaluation.
Train and Evaluate Models:

The train_and_eval function trains and evaluates the model for a specified number of epochs.
It includes the training loop, computing losses, and evaluating metrics.
Training and Evaluating LightGCN:

An instance of RecSysGNN is created for LightGCN.
The optimizer and learnable parameters are configured.
Training and evaluation are performed, and loss and metric values are recorded.
Training and Evaluating NGCF:

Similar to LightGCN, an instance of RecSysGNN is created for NGCF.
The optimizer and learnable parameters are configured.
Training and evaluation are performed, and loss and metric values are recorded.
Comparing Model Performance:

The maximum precision and recall values are compared for both LightGCN and NGCF models.
Generating Movie Recommendations:

The recommend_movies function generates movie recommendations for a specific user.
It computes the relevance scores of all items and selects the top K items.
Output Recommendations:

The recommended movies using both LightGCN and NGCF models are printed for a given user.
Please note that while I've provided explanations for each major section, understanding the full code and its intricacies may require a deeper understanding of PyTorch, PyTorch Geometric, and recommender system techniques.

In [ ]:
# Create LabelEncoder for user IDs
le_user = pp.LabelEncoder()

# Create LabelEncoder for item IDs
le_item = pp.LabelEncoder()

# Encode user IDs and store the indices in 'user_id_idx' column
train_df['user_id_idx'] = le_user.fit_transform(train_df['user_id'].values)

# Encode item IDs and store the indices in 'item_id_idx' column
train_df['item_id_idx'] = le_item.fit_transform(train_df['item_id'].values)


In [ ]:
# Get unique user IDs from the training dataframe
train_user_ids = train_df['user_id'].unique()

# Get unique item IDs from the training dataframe
train_item_ids = train_df['item_id'].unique()

# Print the number of unique user IDs and item IDs
print(len(train_user_ids), len(train_item_ids))

# Filter and update the test dataframe to keep only rows with user and item IDs present in the training data
test_df = test_df[
  (test_df['user_id'].isin(train_user_ids)) & \  # Select rows where user_id is in the list of train_user_ids
  (test_df['item_id'].isin(train_item_ids))    # Select rows where item_id is in the list of train_item_ids
]

# Print the length of the updated test dataframe
print(len(test_df))


In [ ]:
# Transform user IDs in the test dataframe to their corresponding encoded indices using le_user
test_df['user_id_idx'] = le_user.transform(test_df['user_id'].values)

# Transform item IDs in the test dataframe to their corresponding encoded indices using le_item
test_df['item_id_idx'] = le_item.transform(test_df['item_id'].values)


In [ ]:
# Calculate the number of unique user indices in the training data
n_users = train_df['user_id_idx'].nunique()

# Calculate the number of unique item indices in the training data
n_items = train_df['item_id_idx'].nunique()

# Print the number of unique users
print("Number of Unique Users: ", n_users)

# Print the number of unique items
print("Number of Unique Items: ", n_items)


In [ ]:
# Define a function for loading data in batches
def data_loader(data, batch_size, n_usr, n_itm):

    # Define a function to sample negative items
    def sample_neg(x):
        while True:
            # Generate a random negative item index
            neg_id = random.randint(0, n_itm - 1)
            # Check if the negative item index is not in the list of positive items
            if neg_id not in x:
                return neg_id

    # Group the data by user indices and create a list of interacted item indices for each user
    interected_items_df = data.groupby('user_id_idx')['item_id_idx'].apply(list).reset_index()

    # Create a list of indices representing users
    indices = [x for x in range(n_usr)]

    # If the number of users is less than the batch size, select users randomly (with replacement)
    if n_usr < batch_size:
        users = [random.choice(indices) for _ in range(batch_size)]
    # If the number of users is greater than or equal to the batch size, sample users without replacement
    else:
        users = random.sample(indices, batch_size)
    users.sort()

    # Create a DataFrame with the selected user indices
    users_df = pd.DataFrame(users, columns=['users'])

    # Merge the interacted_items_df with the users_df to filter only the selected users' data
    interected_items_df = pd.merge(interected_items_df, users_df, how='right', left_on='user_id_idx', right_on='users')

    # Sample positive items for each selected user randomly from their interacted items
    pos_items = interected_items_df['item_id_idx'].apply(lambda x: random.choice(x)).values

    # Sample negative items for each selected user using the sample_neg function
    neg_items = interected_items_df['item_id_idx'].apply(lambda x: sample_neg(x)).values

    # Return the selected users, positive item indices, and negative item indices as PyTorch LongTensors
    return (
        torch.LongTensor(list(users)).to(device),                  # Selected user indices


In [ ]:
# Convert user indices and adjusted item indices to PyTorch LongTensors
u_t = torch.LongTensor(train_df.user_id_idx)             # User indices
i_t = torch.LongTensor(train_df.item_id_idx) + n_users   # Adjusted item indices (adding n_users to separate from users)

# Create the training edge index tensor by stacking user indices and item indices
# The edge index represents connections between nodes (users and items) in a graph
train_edge_index = torch.stack((
    torch.cat([u_t, i_t]),   # Concatenate user indices and adjusted item indices for the "from" end of edges
    torch.cat([i_t, u_t])    # Concatenate adjusted item indices and user indices for the "to" end of edges
)).to(device)

# Print the resulting training edge index tensor
print(train_edge_index)


In [ ]:
# Extract the last column of the train_edge_index tensor
destination_nodes = train_edge_index[:, -1]  # Extracting the destination (target) nodes (items)

# Extract the first column of the train_edge_index tensor
source_nodes = train_edge_index[:, 0]        # Extracting the source nodes (users)


In [ ]:
edge_index = 0  # Index of the edge you want to retrieve
source_node_index = train_edge_index[:, edge_index][0]  # Index of the source node (user) of the edge
destination_node_index = train_edge_index[:, edge_index][1]  # Index of the destination node (item) of the edge


In [ ]:
It looks like you have created a tensor train_edge_index that represents the edge indices of a graph. The first row of the tensor contains the source node indices (users), and the second row contains the destination node indices (items) for each edge. The output you provided shows that the tensor contains two edges.

Let's break down the outputs you've provided:

train_edge_index[:,-1], train_edge_index[:, 0]: This extracts the last node index of the source nodes and the first node index of the destination nodes from the train_edge_index tensor. This output indicates that the last edge's source node index is 477 and the first edge's destination node index is 64.

train_edge_index[:, len(train)-1], train_edge_index[:, len(train)]: This seems to be a misunderstanding. The len(train)-1 index is used to extract the last column (edge) from the train_edge_index tensor, while len(train) is an invalid index as it's one greater than the last valid index.

If you want to extract specific edges from the train_edge_index tensor, you should provide valid indices for the second dimension (edge index). For example:

python
Copy code
edge_index = 0  # Index of the edge you want to retrieve
source_node_index = train_edge_index[0, edge_index]  # Index of the source node (user) of the edge
destination_node_index = train_edge_index[1, edge_index]  # Index of the destination node (item) of the edge
In your case, train_edge_index[:, -1] and train_edge_index[:, 0] give you the source and destination node indices for the last and first edges, respectively.







In [ ]:
The code defines a custom graph convolutional layer called LightGCNConv using the PyTorch Geometric library. This layer inherits from MessagePassing, and it performs message passing on a graph using a simplified version of the LightGCN algorithm.

The key steps in the code are as follows:

Class Definition: Define the LightGCNConv class inheriting from MessagePassing.

Constructor: Initialize the class by calling the parent class's constructor (super().__init__(aggr='add')).

Forward Method: Define the forward pass of the layer. It takes input node features (x) and edge indices (edge_index) as inputs.

Normalization Calculation: Calculate the degree of each destination node and compute the inverse square root of these degrees for normalization. Handle cases where the degree is infinity by setting it to 0.

Message Passing: Use the inherited propagate method to perform message passing. This method requires specifying the edge_index, input node features (x), and the computed normalization (norm).

Message Method: Define the message computation. Multiply the normalized values (norm) with the source node features (x_j) to scale the features based on the degree-based normalization.

This custom layer can be used as a building block in graph convolutional neural network architectures that follow the LightGCN algorithm. The key idea behind LightGCN is to simplify the message passing step by only considering the degree-based normalization and omitting the use of activation functions and biases.

Remember that this layer is part of a larger framework and is used alongside other components like data loading, model training loops, and evaluation metrics to create a complete recommendation system based on graph neural networks.






In [ ]:
# Import necessary libraries
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import degree

# Define a custom graph convolutional layer class
class LightGCNConv(MessagePassing):
    def __init__(self, **kwargs):
        super().__init__(aggr='add')  # Call parent class constructor with 'add' aggregation method

    def forward(self, x, edge_index):
        # Compute normalization
        from_, to_ = edge_index  # Split edge_index into source and destination nodes
        deg = degree(to_, x.size(0), dtype=x.dtype)  # Calculate degree of each destination node
        deg_inv_sqrt = deg.pow(-0.5)  # Calculate inverse square root of degrees for normalization
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0  # Handle infinity degrees by setting to 0
        norm = deg_inv_sqrt[from_] * deg_inv_sqrt[to_]  # Compute normalization values

        # Start propagating messages (no update after aggregation)
        return self.propagate(edge_index, x=x, norm=norm)

    def message(self, x_j, norm):
        # Multiply normalized values with source node features for message computation
        return norm.view(-1, 1) * x_j


In [ ]:
# Import necessary libraries
import torch
import numpy as np
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import degree

# Define a custom graph convolutional layer class
class LightGCNConv(MessagePassing):
    def __init__(self, **kwargs):
        super().__init__(aggr='add')

    def forward(self, x, edge_index):
        from_, to_ = edge_index
        deg = degree(to_, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[from_] * deg_inv_sqrt[to_]
        return self.propagate(edge_index, x=x, norm=norm)

    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

# Create a test input tensor with shape (5, 5) representing an identity matrix
test_x = torch.Tensor(np.eye(5))

# Create a test edge index tensor with shape (2, 8) representing source and destination nodes
test_edge_index = torch.LongTensor(np.array([
  [0, 0, 1, 1, 2, 3, 3, 4],
  [2, 3, 3, 4, 0, 0, 1, 1]
]))

# Instantiate the LightGCNConv layer and apply it to the test input
output = LightGCNConv()(test_x, test_edge_index)

# Print the output tensor
print(output)


In [ ]:
The code initializes a test input tensor representing an identity matrix and a test edge index tensor representing source and destination nodes. Then, it creates an instance of the LightGCNConv layer and applies it to the test input data using the provided edge index. Finally, it prints the output tensor after the layer's operation.




User
easier way
ChatGPT
Certainly, here's the code with comments to explain each step:

python
Copy code
import torch
import numpy as np
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import degree

# Define a custom graph convolutional layer class
class LightGCNConv(MessagePassing):
    def __init__(self, **kwargs):
        super().__init__(aggr='add')

    def forward(self, x, edge_index):
        from_, to_ = edge_index
        deg = degree(to_, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[from_] * deg_inv_sqrt[to_]
        return self.propagate(edge_index, x=x, norm=norm)

    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

# Create a test input tensor (identity matrix)
test_x = torch.Tensor(np.eye(5))

# Create a test edge index tensor (source and destination nodes)
test_edge_index = torch.LongTensor(np.array([
  [0, 0, 1, 1, 2, 3, 3, 4],
  [2, 3, 3, 4, 0, 0, 1, 1]
]))

# Instantiate the LightGCNConv layer
lightgcn_layer = LightGCNConv()

# Apply the layer to the test input
output = lightgcn_layer(test_x, test_edge_index)

# Print the output
print(output)
This code creates the LightGCNConv layer instance, applies it to the test input data, and prints the output, following the same steps as the previous example but with added comments for clarity.







In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import degree

# Define a graph convolutional layer class called NGCFConv
class NGCFConv(MessagePassing):
    def __init__(self, latent_dim, dropout, bias=True, **kwargs):
        super(NGCFConv, self).__init__(aggr='add', **kwargs)

        # Store dropout rate
        self.dropout = dropout

        # Define two linear layers for message transformation
        self.lin_1 = nn.Linear(latent_dim, latent_dim, bias=bias)
        self.lin_2 = nn.Linear(latent_dim, latent_dim, bias=bias)

        # Initialize layer parameters
        self.init_parameters()

    # Initialize linear layer parameters using Xavier initialization
    def init_parameters(self):
        nn.init.xavier_uniform_(self.lin_1.weight)
        nn.init.xavier_uniform_(self.lin_2.weight)

    # Define the forward pass of the NGCFConv layer
    def forward(self, x, edge_index):
        # Compute normalization coefficients
        from_, to_ = edge_index
        deg = degree(to_, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[from_] * deg_inv_sqrt[to_]

        # Start message propagation and aggregation
        out = self.propagate(edge_index, x=(x, x), norm=norm)

        # Update nodes after aggregation
        out += self.lin_1(x)
        out = F.dropout(out, self.dropout, self.training)
        return F.leaky_relu(out)

    # Define the message computation for each node
    def message(self, x_j, x_i, norm):
        # Transform and combine neighbor node features
        return norm.view(-1, 1) * (self.lin_1(x_j) + self.lin_2(x_j * x_i))

# Code for graph convolutional layer (NGCFConv) is defined above

# Commented explanation:
# This code defines a graph convolutional layer called NGCFConv. It inherits from the MessagePassing class in PyTorch Geometric
# and implements a neighborhood-based collaborative filtering approach.
# The layer takes a latent dimension, dropout rate, and bias as inputs.
# It initializes two linear transformation layers for message passing.
# The init_parameters method initializes the layer parameters using Xavier uniform initialization.
# The forward method computes the normalization coefficients for message propagation.
# It propagates messages and aggregates them using the specified aggregation method.
# The node features are updated after aggregation using the lin_1 layer and dropout.
# The message method computes messages by transforming and combining neighbor node features using lin_1 and lin_2 layers.


In [ ]:
import torch
import torch.nn as nn
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import degree

# Define a custom graph convolutional layer for NGCF model
class NGCFConv(MessagePassing):
    def __init__(self, latent_dim, dropout, bias=True, **kwargs):
        super(NGCFConv, self).__init__(aggr='add', **kwargs)

        self.dropout = dropout

        # Define linear layers for message passing
        self.lin_1 = nn.Linear(latent_dim, latent_dim, bias=bias)
        self.lin_2 = nn.Linear(latent_dim, latent_dim, bias=bias)

        # Initialize parameters
        self.init_parameters()

    # Initialize linear layer parameters using Xavier initialization
    def init_parameters(self):
        nn.init.xavier_uniform_(self.lin_1.weight)
        nn.init.xavier_uniform_(self.lin_2.weight)

    # Define the forward pass for NGCFConv
    def forward(self, x, edge_index):
        # Compute normalization factors
        from_, to_ = edge_index
        deg = degree(to_, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[from_] * deg_inv_sqrt[to_]

        # Start propagating messages
        out = self.propagate(edge_index, x=(x, x), norm=norm)

        # Perform update after aggregation
        out += self.lin_1(x)
        out = F.dropout(out, self.dropout, self.training)
        return F.leaky_relu(out)

    # Define the message function for NGCFConv
    def message(self, x_j, x_i, norm):
        return norm.view(-1, 1) * (self.lin_1(x_j) + self.lin_2(x_j * x_i))

# Define a custom graph neural network model for recommendation
class RecSysGNN(nn.Module):
    def __init__(
        self,
        latent_dim,
        num_layers,
        num_users,
        num_items,
        model,   # Model type: 'NGCF' or 'LightGCN'
        dropout=0.1  # Dropout rate (only used in NGCF)
    ):
        super(RecSysGNN, self).__init__()

        # Check if the model is valid
        assert (model == 'NGCF' or model == 'LightGCN'), \
            'Model must be NGCF or LightGCN'
        self.model = model

        # Create an embedding layer for users and items
        self.embedding = nn.Embedding(num_users + num_items, latent_dim)

        # Initialize convolutional layers based on the chosen model
        if self.model == 'NGCF':
            self.convs = nn.ModuleList(
                NGCFConv(latent_dim, dropout=dropout) for _ in range(num_layers)
            )
        else:
            self.convs = nn.ModuleList(LightGCNConv() for _ in range(num_layers))

        # Initialize parameters
        self.init_parameters()

    # Initialize embedding and convolutional layer parameters
    def init_parameters(self):
        if self.model == 'NGCF':
            nn.init.xavier_uniform_(self.embedding.weight, gain=1)
        else:
            nn.init.normal_(self.embedding.weight, std=0.1)

    # Define the forward pass of the model
    def forward(self, edge_index):
        emb0 = self.embedding.weight
        embs = [emb0]

        emb = emb0
        for conv in self.convs:
            emb = conv(x=emb, edge_index=edge_index)
            embs.append(emb)

        out = (
            torch.cat(embs, dim=-1) if self.model == 'NGCF'
            else torch.mean(torch.stack(embs, dim=0), dim=0)
        )

        return emb0, out

    # Encode a minibatch of data
    def encode_minibatch(self, users, pos_items, neg_items, edge_index):
        emb0, out = self(edge_index)
        return (
            out[users],
            out[pos_items],
            out[neg_items],
            emb0[users],
            emb0[pos_items],
            emb0[neg_items]
        )



Here's the code with comments explaining each part:

python
Copy code
import torch
import torch.nn as nn
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import degree

# Define a custom graph convolutional layer for NGCF model
class NGCFConv(MessagePassing):
    def __init__(self, latent_dim, dropout, bias=True, **kwargs):
        super(NGCFConv, self).__init__(aggr='add', **kwargs)

        self.dropout = dropout

        # Define linear layers for message passing
        self.lin_1 = nn.Linear(latent_dim, latent_dim, bias=bias)
        self.lin_2 = nn.Linear(latent_dim, latent_dim, bias=bias)

        # Initialize parameters
        self.init_parameters()

    # Initialize linear layer parameters using Xavier initialization
    def init_parameters(self):
        nn.init.xavier_uniform_(self.lin_1.weight)
        nn.init.xavier_uniform_(self.lin_2.weight)

    # Define the forward pass for NGCFConv
    def forward(self, x, edge_index):
        # Compute normalization factors
        from_, to_ = edge_index
        deg = degree(to_, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[from_] * deg_inv_sqrt[to_]

        # Start propagating messages
        out = self.propagate(edge_index, x=(x, x), norm=norm)

        # Perform update after aggregation
        out += self.lin_1(x)
        out = F.dropout(out, self.dropout, self.training)
        return F.leaky_relu(out)

    # Define the message function for NGCFConv
    def message(self, x_j, x_i, norm):
        return norm.view(-1, 1) * (self.lin_1(x_j) + self.lin_2(x_j * x_i))

# Define a custom graph neural network model for recommendation
class RecSysGNN(nn.Module):
    def __init__(
        self,
        latent_dim,
        num_layers,
        num_users,
        num_items,
        model,   # Model type: 'NGCF' or 'LightGCN'
        dropout=0.1  # Dropout rate (only used in NGCF)
    ):
        super(RecSysGNN, self).__init__()

        # Check if the model is valid
        assert (model == 'NGCF' or model == 'LightGCN'), \
            'Model must be NGCF or LightGCN'
        self.model = model
        
        # Create an embedding layer for users and items
        self.embedding = nn.Embedding(num_users + num_items, latent_dim)

        # Initialize convolutional layers based on the chosen model
        if self.model == 'NGCF':
            self.convs = nn.ModuleList(
                NGCFConv(latent_dim, dropout=dropout) for _ in range(num_layers)
            )
        else:
            self.convs = nn.ModuleList(LightGCNConv() for _ in range(num_layers))

        # Initialize parameters
        self.init_parameters()

    # Initialize embedding and convolutional layer parameters
    def init_parameters(self):
        if self.model == 'NGCF':
            nn.init.xavier_uniform_(self.embedding.weight, gain=1)
        else:
            nn.init.normal_(self.embedding.weight, std=0.1)

    # Define the forward pass of the model
    def forward(self, edge_index):
        emb0 = self.embedding.weight
        embs = [emb0]

        emb = emb0
        for conv in self.convs:
            emb = conv(x=emb, edge_index=edge_index)
            embs.append(emb)

        out = (
            torch.cat(embs, dim=-1) if self.model == 'NGCF'
            else torch.mean(torch.stack(embs, dim=0), dim=0)
        )

        return emb0, out

    # Encode a minibatch of data
    def encode_minibatch(self, users, pos_items, neg_items, edge_index):
        emb0, out = self(edge_index)
        return (
            out[users],
            out[pos_items],
            out[neg_items],
            emb0[users],
            emb0[pos_items],
            emb0[neg_items]
        )
This code defines a recommendation system using graph neural networks. It includes a custom graph convolutional layer (NGCFConv) for the NGCF model, and a main model class (RecSysGNN) that can be configured to use either NGCF or LightGCN. The code sets up the layers and methods needed for message propagation and parameter initialization, allowing for efficient training and encoding of minibatches of user-item interactions. The model can generate embeddings for users, items, and their interactions, making it suitable for personalized recommendations.

In [ ]:
This code snippet defines a function compute_bpr_loss that computes the Bayesian Personalized Ranking (BPR) loss for a single minibatch of users, positive items, and negative items. The BPR loss is a common loss function used in recommendation systems that aims to optimize the ranking of positive items over negative items for each user. The loss function also includes a regularization term for the initial embeddings.

Here's a breakdown of the code:

python
Copy code
def compute_bpr_loss(users, users_emb, pos_emb, neg_emb, user_emb0, pos_emb0, neg_emb0):
    # Compute regularization loss from initial embeddings
    reg_loss = (1 / 2) * (
        user_emb0.norm().pow(2) +
        pos_emb0.norm().pow(2) +
        neg_emb0.norm().pow(2)
    ) / float(len(users))

    # Compute BPR loss from user, positive item, and negative item embeddings
    pos_scores = torch.mul(users_emb, pos_emb).sum(dim=1)
    neg_scores = torch.mul(users_emb, neg_emb).sum(dim=1)

    # Compute BPR loss using softplus function
    bpr_loss = torch.mean(F.softplus(neg_scores - pos_scores))

    return bpr_loss, reg_loss
In summary, this function computes two parts of the loss:

Regularization Loss (reg_loss): It computes the L2-norm of the initial embeddings (user_emb0, pos_emb0, neg_emb0) and averages them. This term helps prevent overfitting and encourages the embeddings to stay close to the origin.

BPR Loss (bpr_loss): It calculates the BPR loss using the formula for the softplus function applied to the difference between the scores of negative and positive items for each user. This loss aims to maximize the difference in scores between positive and negative items.

The function returns both the BPR loss and the regularization loss, which can then be used to compute the overall loss for training the recommendation system. This loss function is commonly used in recommendation systems to optimize the ranking of items for each user based on their preferences.

The get_metrics function provided appears to calculate the evaluation metrics (Recall@K and Precision@K) for the movie recommendation system based on the provided user and item embeddings, training and test data, and the value of K (number of recommendations to consider). Let's go through the code step by step to understand what it's doing:

Compute Relevance Scores: The function starts by computing the relevance scores for all user-item pairs using the provided user and item embeddings. This is done by matrix multiplication of the user embeddings with the transpose of the item embeddings.

Create Dense Interactions Tensor: It creates a dense tensor interactions_t representing user-item interactions based on the training data. This tensor is created by stacking user indices and item indices from the training data and assigning a value of 1 to each interaction.

Mask Out Training Interactions: The relevance scores are then multiplied element-wise with the complement of interactions_t. This effectively masks out the training interactions from the relevance scores, so that they won't affect the evaluation metrics.

Top-K Scoring Items for Each User: For each user, the code computes the indices of the top K items with the highest relevance scores. These indices represent the recommended items for each user.

Create DataFrames for Results: The indices of the top K items are converted into a DataFrame topk_relevance_indices_df which includes user IDs and the indices of top-K relevant items.

Measure Overlap and Calculate Metrics: The code then calculates the overlap between the recommended items and the held-out (test) user-item interactions. It does this by comparing the recommended item indices with the actual test item indices for each user. The metrics_df DataFrame is constructed to store this information.

Calculate Recall and Precision: The recall and precision are calculated for each user based on the number of overlapping items between recommended items and test items. Recall is the ratio of overlapping items to all test items, and precision is the ratio of overlapping items to K (number of recommendations).

Calculate Mean Metrics: Finally, the function returns the average recall and an undefined variable metric. It seems there might be a typo in the return statement (metric instead of metrics_df).

It's important to note that the evaluation metrics (recall and precision) are calculated for each user, and the overall metrics are reported as the mean values across all users in the test set. These metrics provide insight into how well the recommendation system is performing in terms of capturing relevant items for users based on the provided embeddings and the chosen value of K.

However, the code has a small issue with the return statement, which needs to be fixed to properly return the calculated metrics DataFrame. Also, the provided code snippet is missing the import statements and variable definitions, so make sure you have all the required imports and variable assignments in place for the code to run successfully.






Regenerate


In [ ]:
def get_metrics(user_Embed_wts, item_Embed_wts, n_users, n_items, train_data, test_data, K):
    # Get unique user IDs from the test data
    test_user_ids = torch.LongTensor(test_data['user_id_idx'].unique())

    # Compute the relevance score of all user-item pairs using matrix multiplication
    relevance_score = torch.matmul(user_Embed_wts, torch.transpose(item_Embed_wts, 0, 1))

    # Create a dense tensor representing all user-item interactions from the training data
    i = torch.stack((
        torch.LongTensor(train_data['user_id_idx'].values),
        torch.LongTensor(train_data['item_id_idx'].values)
    ))
    v = torch.ones((len(train_data)), dtype=torch.float64)
    interactions_t = torch.sparse.FloatTensor(i, v, (n_users, n_items)).to_dense().to(device)

    # Mask out training user-item interactions from metric computation
    relevance_score = torch.mul(relevance_score, (1 - interactions_t))

    # Compute the indices of top-scoring items for each user
    topk_relevance_indices = torch.topk(relevance_score, K).indices
    topk_relevance_indices_df = pd.DataFrame(topk_relevance_indices.cpu().numpy(), columns=['top_indx_'+str(x+1) for x in range(K)])
    topk_relevance_indices_df['user_ID'] = topk_relevance_indices_df.index
    topk_relevance_indices_df['top_rlvnt_itm'] = topk_relevance_indices_df[['top_indx_'+str(x+1) for x in range(K)]].values.tolist()
    topk_relevance_indices_df = topk_relevance_indices_df[['user_ID', 'top_rlvnt_itm']]

    # Group test data by user and store their interacted items in a DataFrame
    test_interacted_items = test_data.groupby('user_id_idx')['item_id_idx'].apply(list).reset_index()

    # Merge the top-k recommended items DataFrame with the test interacted items DataFrame
    metrics_df = pd.merge(test_interacted_items, topk_relevance_indices_df, how='left', left_on='user_id_idx', right_on=['user_ID'])

    # Find the intersection of recommended and test interacted items for each user
    metrics_df['intrsctn_itm'] = [list(set(a).intersection(b)) for a, b in zip(metrics_df.item_id_idx, metrics_df.top_rlvnt_itm)]

    # Calculate recall and precision for each user
    metrics_df['recall'] = metrics_df.apply(lambda x: len(x['intrsctn_itm']) / len(x['item_id_idx']), axis=1)
    metrics_df['precision'] = metrics_df.apply(lambda x: len(x['intrsctn_itm']) / K, axis=1)

    # Calculate the mean recall across all users and return the metrics DataFrame
    return metrics_df['recall'].mean(), metrics_df['precision'].mean()


In [ ]:
It looks like this code defines a training and evaluation loop for a recommender system model using Bayesian Personalized Ranking (BPR) loss. Let's go through the code step by step and provide explanations for each part:

python
Copy code
latent_dim = 64
n_layers = 3

EPOCHS = 50
BATCH_SIZE = 1024
DECAY = 0.0001
LR = 0.005
K = 20
Here, several hyperparameters are defined:

latent_dim: The dimensionality of the learned embeddings for users and items.
n_layers: The number of layers in the neural network model.
EPOCHS: The number of training epochs.
BATCH_SIZE: The batch size used during training.
DECAY: The weight decay factor used for regularization.
LR: The learning rate of the optimizer.
K: The number of top items to consider for evaluation metrics like recall and precision.
python
Copy code
def train_and_eval(model, optimizer, train_df):
    # Lists to store different types of losses and evaluation metrics for each epoch
    loss_list_epoch = []
    bpr_loss_list_epoch = []
    reg_loss_list_epoch = []
    recall_list = []
    precision_list = []

    for epoch in tqdm(range(EPOCHS)):
        n_batch = int(len(train_df) / BATCH_SIZE)

        final_loss_list = []
        bpr_loss_list = []
        reg_loss_list = []

        model.train()
        for batch_idx in range(n_batch):
            optimizer.zero_grad()

            # Sample users, positive items, and negative items for the batch
            users, pos_items, neg_items = data_loader(train_df, BATCH_SIZE, n_users, n_items)

            # Encode the minibatch to obtain user and item embeddings
            users_emb, pos_emb, neg_emb, userEmb0,  posEmb0, negEmb0 = model.encode_minibatch(users, pos_items, neg_items, train_edge_index)

            # Compute BPR loss and regularization loss
            bpr_loss, reg_loss = compute_bpr_loss(
                users, users_emb, pos_emb, neg_emb, userEmb0,  posEmb0, negEmb0
            )
            reg_loss = DECAY * reg_loss
            final_loss = bpr_loss + reg_loss

            # Backpropagate and update the model parameters
            final_loss.backward()
            optimizer.step()

            # Append different types of losses to lists
            final_loss_list.append(final_loss.item())
            bpr_loss_list.append(bpr_loss.item())
            reg_loss_list.append(reg_loss.item())

        # Evaluate the model on the test data
        model.eval()
        with torch.no_grad():
            _, out = model(train_edge_index)
            final_user_Embed, final_item_Embed = torch.split(out, (n_users, n_items))
            test_topK_recall,  test_topK_precision = get_metrics(
                final_user_Embed, final_item_Embed, n_users, n_items, train_df, test_df, K
            )

        # Append losses and evaluation metrics for the current epoch
        loss_list_epoch.append(round(np.mean(final_loss_list), 4))
        bpr_loss_list_epoch.append(round(np.mean(bpr_loss_list), 4))
        reg_loss_list_epoch.append(round(np.mean(reg_loss_list), 4))
        recall_list.append(round(test_topK_recall, 4))
        precision_list.append(round(test_topK_precision, 4))

    # Return the lists of losses and evaluation metrics for all epochs
    return (
        loss_list_epoch,
        bpr_loss_list_epoch,
        reg_loss_list_epoch,
        recall_list,
        precision_list
    )
In summary, the train_and_eval function trains a recommender system model for a specified number of epochs and performs evaluation after each epoch. During training, the model is trained using BPR loss, which aims to maximize the margin between the scores of positive and negative items for each user. The model is then evaluated on the test data using the get_metrics function to compute recall and precision metrics for the top-K recommendations.

This code is part of a larger training and evaluation pipeline for a recommender system model. It assumes that other functions like data_loader, compute_bpr_loss, and get_metrics are defined elsewhere in the codebase and are used to handle data loading, loss computation, and evaluation.






Regenerate


In [ ]:
t looks like you are training and evaluating a LightGCN model for a recommendation task. The code snippet you provided is creating an instance of the RecSysGNN model with the "LightGCN" architecture, moving it to the specified device (presumably GPU), setting up the optimizer, and then training and evaluating the model using the train_and_eval function. Let's break down the steps:

Model Initialization:

python
Copy code
lightgcn = RecSysGNN(
  latent_dim=latent_dim,
  num_layers=n_layers,
  num_users=n_users,
  num_items=n_items,
  model='LightGCN'
)
lightgcn.to(device)
Here, a RecSysGNN model with the "LightGCN" architecture is initialized. The model's hyperparameters such as latent_dim, num_layers, num_users, and num_items are provided. The model is then moved to the specified device (GPU) using .to(device).

Optimizer Setup:

python
Copy code
optimizer = torch.optim.Adam(lightgcn.parameters(), lr=LR)
An Adam optimizer is set up to optimize the parameters of the lightgcn model. The learning rate LR is used as defined in your previous code.

Print Learnable Embedding Size:

python
Copy code
print("Size of Learnable Embedding : ", [x.shape for x in list(lightgcn.parameters())])
This code snippet prints the sizes of learnable embeddings in the lightgcn model. In this case, it prints the size of the embedding for users, which is [torch.Size([706, 64])], indicating that there are 706 users and each user's embedding has a dimension of 64.

Training and Evaluation:

python
Copy code
light_loss, light_bpr, light_reg, light_recall, light_precision = train_and_eval(lightgcn, optimizer, train_df)
The train_and_eval function is called with the lightgcn model, the optimizer, and the training data (train_df). The function returns various loss values (light_loss, light_bpr, light_reg), as well as recall and precision metrics (light_recall, light_precision), which are all assigned to variables for later analysis.

Overall, this code snippet initializes, trains, and evaluates a LightGCN-based recommender system model, recording various loss values and evaluation metrics during the training process.





Regenerate


In [ ]:
This code appears to be responsible for visualizing the training and evaluation results of two different recommender models: LightGCN and NGCF. Let's go through it step by step:

Setting Up Epoch List:

python
Copy code
epoch_list = [(i+1) for i in range(EPOCHS)]
An epoch_list is created containing the numbers from 1 to EPOCHS, representing the x-axis values for plotting.

Plotting LightGCN Training Losses:

python
Copy code
plt.plot(epoch_list, light_loss, label='Total Training Loss')
plt.plot(epoch_list, light_bpr, label='BPR Training Loss')
plt.plot(epoch_list, light_reg, label='Reg Training Loss')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
This code segment creates a plot with three lines representing different types of loss during the training of the LightGCN model: total training loss, Bayesian Personalized Ranking (BPR) training loss, and regularization training loss. The x-axis corresponds to epochs, and the y-axis corresponds to loss values. Labels and legends are added to differentiate the lines.

Plotting LightGCN Recall and Precision Metrics:

python
Copy code
plt.plot(epoch_list, light_recall, label='Recall')
plt.plot(epoch_list, light_precision, label='Precision')
plt.xlabel('Epoch')
plt.ylabel('Metrics')
plt.legend()
Similar to the previous code segment, this part creates a plot for recall and precision metrics obtained during the training of the LightGCN model. The x-axis represents epochs, and the y-axis represents metric values. Legends are added to differentiate the lines.

Creating NGCF Model:

python
Copy code
ngcf = RecSysGNN(
  latent_dim=latent_dim,
  num_layers=n_layers,
  num_users=n_users,
  num_items=n_items,
  model='NGCF'
)
ngcf.to(device)
An instance of the RecSysGNN model with the "NGCF" architecture is created and moved to the specified device (GPU).

Optimizer Setup for NGCF:

python
Copy code
optimizer = torch.optim.Adam(ngcf.parameters(), lr=LR)
An optimizer is set up to optimize the parameters of the NGCF model.

Printing Learnable Embedding Size for NGCF:

python
Copy code
print("Size of Learnable Embedding : ", [x.shape for x in list(ngcf.parameters())])
This code prints the sizes of learnable embeddings in the NGCF model.

Training and Evaluating NGCF:

python
Copy code
ngcf_loss, ngcf_bpr, ngcf_reg, ngcf_recall, ngcf_precision = train_and_eval(ngcf, optimizer, train_df)
The NGCF model is trained and evaluated using the train_and_eval function, and the obtained loss and metric values are assigned to the corresponding variables.

Plotting NGCF Training Losses:

python
Copy code
plt.plot(epoch_list, ngcf_loss, label='Total Training Loss')
plt.plot(epoch_list, ngcf_bpr, label='BPR Training Loss')
plt.plot(epoch_list, ngcf_reg, label='Reg Training Loss')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
This section plots the training losses (total, BPR, and regularization) during the training of the NGCF model.

Plotting NGCF Recall and Precision Metrics:

python
Copy code
plt.plot(epoch_list, ngcf_recall, label='Recall')
plt.plot(epoch_list, ngcf_precision, label='Precision')
plt.xlabel('Epoch')
plt.ylabel('Metrics')
plt.legend()
Similar to the previous part, this section creates a plot for recall and precision metrics obtained during the training of the NGCF model.

In summary, this code snippet visualizes the training and evaluation results for both the LightGCN and NGCF models. It produces plots to show how the losses and evaluation metrics change over the course of training.





Regenerate


In [ ]:
The code snippet you provided seems to be focused on making movie recommendations using the trained LightGCN and NGCF models. It calculates and displays the recommended movies for a specific user based on their embeddings and the relevance scores of items. Let's go through the code step by step:

Calculating Maximum Precision and Recall:

python
Copy code
max(light_precision), max(light_recall)
This code calculates and displays the maximum precision and recall values obtained during the training of the LightGCN model.

python
Copy code
max(ngcf_precision), max(ngcf_recall)
Similarly, this code calculates and displays the maximum precision and recall values obtained during the training of the NGCF model.

Defining the recommend_movies Function:

python
Copy code
def recommend_movies(user_id, user_Embed_wts, item_Embed_wts, n_items, K):
    # ... (function implementation)
This function takes a user ID, user embeddings, item embeddings, the total number of items, and the number of recommendations to generate as arguments. It returns a list of recommended movie IDs for the specified user.

Generating Recommendations for LightGCN and NGCF Models:

python
Copy code
user_id = 122  # Replace with the specific user ID for which you want recommendations
recommended_movies_lightgcn = recommend_movies(user_id, lightgcn.embedding.weight, lightgcn.embedding.weight[n_users:], n_items, K)
recommended_movies_ngcf = recommend_movies(user_id, ngcf.embedding.weight, ngcf.embedding.weight[n_users:], n_items, K)
These lines of code use the recommend_movies function to generate movie recommendations for the specified user ID using both the LightGCN and NGCF models. The recommendations are stored in the variables recommended_movies_lightgcn and recommended_movies_ngcf.

Printing Recommended Movies:

python
Copy code
print("Recommended movies (LightGCN):", recommended_movies_lightgcn)
print("Recommended movies (NGCF):", recommended_movies_ngcf)
These lines print out the recommended movies for the specified user using both the LightGCN and NGCF models.

The overall purpose of this code is to demonstrate how to use the trained recommendation models to generate personalized movie recommendations for a specific user and then display those recommendations. The recommend_movies function computes relevance scores for items, sorts them, and returns the top K recommendations. The code snippet then showcases this functionality using the LightGCN and NGCF models.





Regenerate


Based on the code provided, I can explain the technical process of why movies were recommended to a user and the metrics used to arrive at these recommendations.

Generating Movie Recommendations:

The process of generating movie recommendations is based on the embeddings of users and items (movies) learned by the recommendation models (LightGCN and NGCF). These embeddings capture the latent features of users and items, allowing the models to understand the preferences and characteristics of both.

The recommend_movies function takes the following inputs:

user_id: The ID of the user for whom recommendations are being generated.
user_Embed_wts: Embeddings of all users.
item_Embed_wts: Embeddings of all items (movies).
n_items: The total number of items (movies) in the dataset.
K: The number of recommendations to generate.
Here's how the process works:

The function first retrieves the embedding of the specified user using their user_id.
It calculates the relevance score between the user's embedding and the embeddings of all items (movies) using matrix multiplication.
The items are then ranked based on their relevance scores.
The top K items with the highest relevance scores are selected as recommendations for the user.
Metrics and Thresholds:

From the provided code, the metrics used to evaluate the quality of recommendations are "precision" and "recall." These metrics help assess how well the recommended movies match the user's actual interactions or preferences.

Precision: This metric measures the proportion of recommended items that the user has actually interacted with or liked. It's calculated as the ratio of the number of recommended items that the user has interacted with to the total number of recommended items. Higher precision indicates that the recommendations are closely aligned with the user's preferences.

Recall: This metric measures the proportion of the user's interacted items that were successfully recommended. It's calculated as the ratio of the number of recommended items that the user has interacted with to the total number of items that the user has interacted with. Higher recall indicates that the recommendations are comprehensive and capture a significant portion of the user's preferences.

While the exact threshold values for precision and recall are not explicitly mentioned in the provided code, typically there are no fixed "one-size-fits-all" threshold values. The choice of threshold values can depend on the specific use case, user behavior, and trade-offs between precision and recall. However, in general, higher values for both precision and recall are desirable.

It's important to note that evaluating the quality of recommendations is an ongoing process, and different models and algorithms might have varying precision-recall trade-offs. The chosen metrics and thresholds should align with the specific goals of the recommendation system and the preferences of the users.






Certainly, let's delve into the mathematical reasoning behind the metrics and thresholds used to arrive at movie recommendations, specifically precision and recall.

Precision:

Precision is defined mathematically as the ratio of the number of relevant items retrieved (recommended) to the total number of items retrieved (recommended). In the context of movie recommendations:

Precision
=
Number of Recommended Relevant Movies
Total Number of Recommended Movies
Precision=
Total Number of Recommended Movies
Number of Recommended Relevant Movies
​


The numerator represents the count of movies in the recommendations that the user has actually interacted with (relevant movies).
The denominator represents the total number of movies recommended to the user.
The higher the precision, the more accurate the recommendations are in terms of aligning with the user's preferences. A precision of 1.0 indicates that all recommended movies are relevant to the user.

Recall:

Recall is defined mathematically as the ratio of the number of relevant items retrieved (recommended) to the total number of relevant items (those actually interacted with). In the context of movie recommendations:

Recall
=
Number of Recommended Relevant Movies
Total Number of Relevant Movies
Recall=
Total Number of Relevant Movies
Number of Recommended Relevant Movies
​


The numerator represents the count of movies in the recommendations that the user has actually interacted with (relevant movies).
The denominator represents the total number of movies that the user has interacted with.
The higher the recall, the more comprehensive the recommendations are in terms of capturing the user's preferences. A recall of 1.0 indicates that all the user's interacted movies are included in the recommendations.

Thresholds:

Thresholds for precision and recall are typically not explicitly defined in the code but rather set by the system designers based on their goals and user behavior. Here's how they can be understood mathematically:

Precision Threshold: If you want to ensure that a certain proportion of the recommended movies are relevant, you can set a precision threshold. For example, a precision threshold of 0.8 means you want at least 80% of the recommended movies to be relevant.

Recall Threshold: If you want to ensure that a certain proportion of the user's interacted movies are covered by the recommendations, you can set a recall threshold. For example, a recall threshold of 0.7 means you want to capture at least 70% of the user's relevant movies.

Adjusting these thresholds allows you to balance the trade-off between precision and recall based on the specific needs of your recommendation system and the preferences of your users. It's important to note that precision and recall are inversely related; as one increases, the other may decrease. The choice of thresholds should consider the user experience and the system's objectives.





Regenerate
